In [13]:
import numpy as np
import pandas as pd


In [14]:
# get data from csv file
thread_df = pd.read_csv('data/uk_subreddit_threads.csv')
comment_df = pd.read_csv('data/uk_subreddit_comments.csv')


In [15]:
thread_df

,title,score,id,url,comms_num,created,body
0,"In the middle of a desert, almost 100 miles fr...",793,150lepk,https://www.reddit.com/r/electricvehicles/comm...,310,1.689451e+09,So I was driving across the desert of souther...
1,Tesla gives new owners in the UK a grabbing st...,353,14n2lrr,https://electrek.co/2023/06/30/tesla-gives-new...,288,1.688137e+09,NaN
2,New Volvo EX30 Cross Country will be coming to...,35,14ygkih,https://www.autocar.co.uk/car-news/new-cars/ne...,11,1.689242e+09,NaN
3,"New 2023 Kia EV9 priced from £64,995 in the UK...",10,14r9oft,https://www.autocar.co.uk/car-news/new-cars/ne...,27,1.688561e+09,NaN
4,Polestar 2 Hertz Rental UK review,15,14m9x2u,https://www.reddit.com/r/electricvehicles/comm...,24,1.688057e+09,I spent a week in the UK with a Polestar 2 tha...
...,...,...,...,...,...,...,...
556,Why Northern Ireland is Mess Over E-Bike Regul...,11,6uf36o,http://www.bike-eu.com/laws-regulations/nieuws...,3,1.503025e+09,NaN
557,Northern Ireland: Wrightbus debuts world’s fir...,6,9t2qel,https://www.electrichybridvehicletechnology.co...,0,1.541019e+09,NaN
558,China’s BYD blames Brexit as it rules out UK f...,368,11plpmc,https://www.theguardian.com/business/2023/mar/...,83,1.678646e+09,NaN
559,BEV market share in Northern Europe vs. Southe...,5,ywqobd,https://ecomento.de/wp-content/uploads/2022/11...,40,1.668597e+09,NaN


In [16]:
comment_df
# rename submission_id column to thread_id
comment_df = comment_df.rename(columns={'submission_id': 'thread_id'})
comment_df

,body,score,created,id,thread_id,location
0,Checked supercharge.info\n\nUnited Kingdom 118...,703,2023-07-15 21:19:43,js3upk8,150lepk,js3upk8
1,I’m in England. I have an EV. \n\nToday and to...,112,2023-07-15 22:03:13,js40lpa,150lepk,js40lpa
2,Lol.\n\nThere's definitely a lot of people ove...,106,2023-07-15 21:13:16,js3ttv8,150lepk,js3ttv8
3,...must have been a Brexiteer. How's that going?,166,2023-07-15 21:14:11,js3tyf4,150lepk,js3tyf4
4,A guy stopped me outside the store in Connecti...,25,2023-07-15 23:06:22,js48xjo,150lepk,js48xjo
...,...,...,...,...,...,...
21164,"Might be banning domestic flights like France,...",1,2021-07-16 18:06:14,h5f1rhk,ok1ej7,h5f1rhk
21165,Even worse is that they are building a new non...,1,2021-07-16 18:08:21,h5f222f,ok1ej7,h5f222f
21166,Yes,1,2021-07-15 08:29:46,h590y4x,ok1ej7,h590y4x
21167,"Agreed. Full hydrogen is (today, and especiall...",5,2021-07-14 15:00:45,h55l5dn,ok1ej7,h55l5dn
